In [1]:
import sys
import os
import cv2
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import random
import matplotlib.pyplot as plt
from numpy import pi, matmul,sqrt,dot,array,zeros,cos,sin,pi,arccos
from torch.utils.data import DataLoader
from func import OR,heatplot,misorientation,mat2plot,imgshow,ipfread,negsample,create_dataset
from Class import Data,Cluster,Dataset,CNN3,CNN5
import torch.nn.functional as F
import matplotlib.patches as patches

In [2]:
train=create_dataset("data/train/",feature=["Orient","BC","BS"])
test=create_dataset("data/test/",feature=["Orient","BC","BS"])

In [4]:
train_loader= DataLoader(train, batch_size=64, shuffle=True,  num_workers=0, drop_last=True )
test_loader= DataLoader(test, batch_size=64, shuffle=True,  num_workers=0,  drop_last=False )

In [10]:
for __ in range(10):
    in_channel=train[0][0].shape[0]
    model = CNN3()
    Loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    max_acc = 0.7
    epoch = 200
    for ep in range(epoch):
        for batch_ndx, sample in enumerate(train_loader):
            optimizer.zero_grad()
            X,Y,_= sample
            output=model(X)
            loss = Loss(output,Y)
            loss.backward()
            optimizer.step()
    #         print("train ",ep,loss.item())
        model.eval()
        cum=0
        loss=0
        for batch_ndx, sample in enumerate(test_loader):
            X,Y,_= sample
            output=model(X)
            loss+=Loss(output,Y)
            predict = torch.max(output, 1)[1]
            cum+=np.sum((Y == predict).cpu().numpy())
        acc=cum/len(test)
        if acc>max_acc:
            max_acc=acc
            min_loss = loss.item()
            print(ep,min_loss)
            print("acc= ",cum,"/",len(test))
            torch.save(model,"model/%s_obcbs_%.3f.pkl"%(type(model).__name__,acc))
        
    

40 1.2085299491882324
acc=  73 / 102
51 1.2058465480804443
acc=  75 / 102
75 1.154876708984375
acc=  76 / 102
81 1.1655160188674927
acc=  77 / 102


KeyboardInterrupt: 

In [15]:
train[3][2]

['20200916', (30, 387)]